## PACE Rapid Response Workflow

#### Basic Steps
- Use worldview to identify an event and region of interest to examine (basic before/after)
- define lat lon bounds, and time range for pre- and post-event averages
- earthdata search using parameters. Open the first one by itself to establish gridding params
- use dask to open all the pre-event granules
- use dask to open all the post-event granules
- make some initial analysis plots
- return pre and post dataframes to user, to allow them to make more custom plots

In [1]:
import cartopy.crs as ccrs
import earthaccess
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from matplotlib.colors import LogNorm
import cmocean
from dask.distributed import Client
from matplotlib.patches import Rectangle

/Users/mkehrli/miniforge3/envs/hackweek/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Case Study

Hurricane Erin, look at chl-a before and after hurricane off FL coast
- as Carlos mentioned last meeting, hurricanes tend to increase productivity
Timeline (using worldview): 
- Hurricane off florida/gulf stream PACE imagery on AUG 20
- Aug 21, hurricane passed and clear imagery
- Aug 18/19 hurricane not there yet

*post Aug 21, there is a PACE safehold so no data. But, the imagery from Aug21 immediately post hurricane is enough for this example script

In [2]:
# User definitions:

pre_tspan = ("2025-08-18", "2025-08-20")
post_tspan = ("2025-08-21", "2025-08-22")

min_lon = -82
min_lat = 27.8
max_lon = -70
max_lat = 32.1

suite_name="PACE_OCI_L2_BGC_NRT" # oci suite name
var_name = "chlor_a"

##
bbox = (min_lon, min_lat, max_lon, max_lat)

In [3]:
# Define Functions
def grid_match(path, dst_crs, dst_shape, dst_transform):
    """Reproject a Level-2 granule to match a Level-3M-ish granule."""
    dt = xr.open_datatree(path)
    da = dt["geophysical_data"]["chlor_a"]
    da = da.rio.set_spatial_dims("pixels_per_line", "number_of_lines")
    da = da.rio.set_crs("epsg:4326")
    da = da.rio.reproject(
        dst_crs,
        shape=dst_shape,
        transform=dst_transform,
        src_geoloc_array=(
            dt["navigation_data"]["longitude"],
            dt["navigation_data"]["latitude"],
        ),
    )
    da = da.rename({"x":"longitude", "y":"latitude"})
    return da

def time_from_attr(ds):
    """Set the start time attribute as a dataset variable.
 
    Parameters
    ----------
    ds
        a dataset corresponding to a Level-2 granule
    """
    datetime = ds.attrs["time_coverage_start"].replace("Z", "")
    ds["time"] = ((), np.datetime64(datetime, "ns"))
    ds = ds.set_coords("time")
    return ds

In [ ]:
'''One large cell to load in all relevant pre-event and post-event granules
Steps:
- Earth access search to make lists of pre and post event files
- Open first entry of list manually, manipulate to L3M-like format and store relevant grid configuration info
- Using dask, open pre and post event granules, recast to L3M-like mapped format, trim to spatial extent, combine 2 datasets
- Make some default plots to visualize Pre and Post event data
'''

pre_results = earthaccess.search_data(
    short_name=suite_name,
    temporal=pre_tspan,
    bounding_box=bbox,# logic to select central point of bounding box
)
#pre_paths = earthaccess.open(pre_results) # MK changed to below to test download
pre_paths = earthaccess.download(pre_results, local_path="data")
print(" Number of pre-event granules: "+str(len(pre_results)))

post_results = earthaccess.search_data(
    short_name=suite_name,
    temporal=post_tspan,
    bounding_box=bbox,# logic to select central point of bounding box
)
#post_paths = earthaccess.open(post_results) # MK changed to below to test download
post_paths = earthaccess.download(post_results, local_path="data")
print(" Number of post-event granules: "+str(len(post_results)))

# Part 1: load first file in pre_paths, manipulate to L3M-like, store geo info

datatree = xr.open_datatree(pre_paths[0])
dataset = xr.merge(datatree.to_dict().values())
dataset = dataset.set_coords(("longitude", "latitude"))

var_data = dataset[var_name]# use code from dask_gridding notebook to transform L2 granule to L3M-like grid
var_data = var_data.rio.set_spatial_dims("pixels_per_line", "number_of_lines")
var_data = var_data.rio.write_crs("epsg:4326")
var_L3M = var_data.rio.reproject(
    dst_crs="epsg:4326",
    src_geoloc_array=(
        var_data.coords["longitude"],
        var_data.coords["latitude"],
    ),
)
var_L3M = var_L3M.rename({"x":"longitude", "y":"latitude"})

var_L3M_aoi = var_L3M.sel({"longitude": slice(bbox[0], bbox[2]),"latitude": slice(bbox[3], bbox[1])})

crs = var_L3M_aoi.rio.crs# set mapping parameters from newly transformed file, to use when opening the rest with dask
shape = var_L3M_aoi.rio.shape
transform = var_L3M_aoi.rio.transform()

# part 2: using dask, open all the data in pre_paths and post_paths
# 2a load pre event files
client = Client()
pre_futures = client.map(grid_match,pre_paths,dst_crs=crs,dst_shape=shape,dst_transform=transform)
kwargs = {"combine": "nested", "concat_dim": "time"}
pre_attrs = xr.open_mfdataset(pre_paths, preprocess=time_from_attr, **kwargs)
pre_data = xr.combine_nested(client.gather(pre_futures), concat_dim="time")# open  4 files. they are stored in same xarray dataset at different "time" coordinates 
pre_data["time"] = pre_attrs["time"]
#client.close()
print('loaded pre event files')

# 2b post event files
client = Client()
post_futures = client.map(grid_match,post_paths,dst_crs=crs,dst_shape=shape,dst_transform=transform)
kwargs = {"combine": "nested", "concat_dim": "time"}
post_attrs = xr.open_mfdataset(post_paths, preprocess=time_from_attr, **kwargs)
post_data = xr.combine_nested(client.gather(post_futures), concat_dim="time")# open a 4 files. they are stored in same xarray dataset at different "time" coordinates 
post_data["time"] = post_attrs["time"]
client.close()
print('loaded post event files')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 6), subplot_kw={'projection': ccrs.PlateCarree()})
ax[0].gridlines(draw_labels={"left": "y", "bottom": "x"})
plot = pre_data.mean("time").plot(x="longitude", y="latitude" , cbar_kwargs={'label': 'Chlor mg/m3', 'shrink':0.5}, cmap=cmocean.cm.haline,norm=LogNorm(vmin=.01, vmax=5),  ax=ax[0], robust=True)
ax[0].set_xlim(bbox[0]-3,bbox[2]+3,)
ax[0].set_ylim(bbox[1]-3,bbox[3]+3)
ax[0].coastlines()
ax[0].set_title('pre_hurricane_8/18 & 8/19')


ax[1].gridlines(draw_labels={"left": "y", "bottom": "x"})
plot = post_data.mean("time").plot(x="longitude", y="latitude", cbar_kwargs={'label': 'Chlor mg/m3', 'shrink':0.5}, cmap=cmocean.cm.haline,norm=LogNorm(vmin=.01, vmax=5),  ax=ax[1], robust=True)
ax[1].set_xlim(bbox[0]-3,bbox[2]+3,)
ax[1].set_ylim(bbox[1]-3,bbox[3]+3)
ax[1].coastlines()
ax[1].set_title('post_hurricane 8/21')


In [ ]:
# compare the pre and post event data. first, mask both to only pixels where data is available before and after the hurricane
# then, mask each where it's values are greater or less than

pre_mean = pre_data.mean("time")
post_mean = post_data.mean("time")

post_mean['longitude'] = post_mean['longitude'].round(5)# round lat/lon to 5 decimals, was having issues with floats at large # of decimals not being exactly the same
post_mean['latitude'] = post_mean['latitude'].round(5)
pre_mean['longitude'] = pre_mean['longitude'].round(5)
pre_mean['latitude'] = pre_mean['latitude'].round(5)

# Create a mask where both datasets have valid values
mask = ~np.isnan(post_mean) & ~np.isnan(pre_mean)

# Apply mask to both datasets
post_mean_mask = post_mean.where(mask)
pre_mean_mask = pre_mean.where(mask)

# new mask to show where post > pre and vice versa
mask2 = post_mean<pre_mean
mask3 = post_mean>pre_mean

# Apply mask to both datasets
pre_mean_mask = pre_mean_mask.where(mask2)# locations where chla is higher pre hurricane
post_mean_mask = post_mean_mask.where(mask3)# locations where chla is higher post hurricane


fig, ax = plt.subplots(1, 2, figsize=(15, 6), subplot_kw={'projection': ccrs.PlateCarree()})
ax[0].gridlines(draw_labels={"left": "y", "bottom": "x"})
plot = pre_mean_mask.plot(x="longitude", y="latitude" , cbar_kwargs={'label': 'Chlor mg/m3', 'shrink':0.5}, cmap=cmocean.cm.haline,norm=LogNorm(vmin=.01, vmax=5),  ax=ax[0], robust=True)
ax[0].set_xlim(bbox[0]-3,bbox[2]+3,)
ax[0].set_ylim(bbox[1]-3,bbox[3]+3)
ax[0].coastlines()
ax[0].set_title('pre_hurricane higher chla')

ax[1].gridlines(draw_labels={"left": "y", "bottom": "x"})
plot = post_mean_mask.plot(x="longitude", y="latitude", cbar_kwargs={'label': 'Chlor mg/m3', 'shrink':0.5}, cmap=cmocean.cm.haline,norm=LogNorm(vmin=.01, vmax=5),  ax=ax[1], robust=True)
ax[1].set_xlim(bbox[0]-3,bbox[2]+3,)
ax[1].set_ylim(bbox[1]-3,bbox[3]+3)
ax[1].coastlines()
ax[1].set_title('post_hurricane higher chla')
